![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)


## spatial: Spatial Position Sensor

#### January 2024

This short notebook is a minimal demonstration of how to access a Phidget device from a Jupyter notebook connect it to Python code.

It is important to run this notebook in a suitable  web browser: Google Chrome, Chromium or MS Edge. Unfortunately, other browsers do not currently support the WebUSB protocol for access to USB devices from a web page. 

## Setting up the Phidget device

You will need a Phidget VINT hub (HUB0001) and a Phidget Spatiall Sensor (MOT1102) connected together as shown here:
<p style="text-align:center;">
<img src="images/SpatialSensor.png" alt="Image of the spatial sensor and VINT Hub"  width = 400 />
</p>
<p style="text-align:center;">
Phidget VINT Hub and Spatial Sensor connected together. (Photo Credits: Phidgets Inc.)
</p>

Use a USB cable to connect the VINT Hub to your computer. 

If you have an older version of the Phidgets, you may need to update its firmware. See the following for details:

- https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/

### Notes on local and remote computer

Note we are using two computers here. There is the laptop (or desktop) computer sitting in front of you, which is the **local** computer. The Callysto Hub, that you're running this notebook on, is a piece of software running on a cloud-based network computer located at the Cybera offices in Alberta (usually), and we call this the **remote** computer. 

We run Python code on the Callysto Hub, while the Phidget device is accessed from the local computer using Javascript. JavaScript is just another programming language, but with a syntax and structure somewhat different than Python. You will see Javascript in the code below as part of the widget class.

For more information on how to write Javascript for accessing the Phidget Spatial Sensor device, please refer to the Phidgets webpages:

- https://www.phidgets.com/?prodid=1252#Tab_API

Run the following cells and follow the instructions at the end. 

In [1]:
import piplite
await piplite.install(['ipywidgets','anywidget'])
import anywidget
import traitlets
from ipywidgets import Box, FloatProgress

In [2]:
## Create the spatial widget class, which communicates with the Phidget devices.

class spatial_widget(anywidget.AnyWidget):
    _esm = """

import {USBConnection, Spatial} from "https://esm.sh/phidget22@3.17";

export function render({ model, el }) {
    var conn = 0; // the USB connection
    var chan = 0; // the channel for the device
    var isOpen = false;
    var connOpen = false;
    var chanOpen = false;

    async function openUSB() {
        if (conn) {
            try {await chan.close();} catch {}
            try {await conn.close();} catch {}
            try {await conn.delete();} catch {}
        }
        conn = new USBConnection();
        chan = new Spatial();
    	chan.onAlgorithmData = function(value, timestamp) {
            textValue.innerHTML = 'Position is ' + 
                value[0].toFixed(4) + ' ' + 
                value[1].toFixed(4) + ' ' + 
                value[2].toFixed(4) + ' ' + 
                value[3].toFixed(4) + ' ';
            model.set('value', value);
            model.save_changes();
    	};
        chan.onAttach = async () => {
            textStatus.innerHTML = 'Attached. ';
            await chan.setDataInterval(250);
        }

        try {
    		await conn.connect();
            conn.requestWebUSBDeviceAccess();
            connOpen = true;
    	} catch(err) {connOpen = false;}
        if (connOpen) {
            chanOpen = false;
            try {
                await chan.open(2000);
                chanOpen = true;
            } catch(err) {chanOpen = false;}
        }
        
        // now let the UI reflect the status of the connections
        if (connOpen && chanOpen) {
            isOpen = true;
            textStatus.innerHTML = 'USB connected, channel open. ';
            button.innerHTML = `Click to disconnect`;
        }
        if (connOpen && !chanOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB connected, channel not open. ';
            button.innerHTML = `Click to connect`;
        }
        if (!connOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB did not connect. ';
            button.innerHTML = `Click to connect`;        
        }
    };
    async function closeUSB() {
        try {await chan.close();} catch {}
        try {await conn.close();} catch {}
        try {await conn.delete();} catch {}
        isOpen = false; 
        connOpen = false;
        chanOpen = false;
        textStatus.innerHTML = 'Disconnected. ';
        textValue.innerHTML = 'Position is null. ';
        button.innerHTML = `Click to connect`;
    };
    
    // here we define the user interface, a button and two text boxes
    let button = document.createElement("button");
    button.classList.add("ph-button");
    button.innerHTML = `Click to open USB`;
    button.addEventListener("click", async () => {
        if (isOpen) {closeUSB();} else {openUSB();}
    });
    let textStatus = document.createElement("label");
    textStatus.innerHTML = 'Status message here. ';
    let textValue = document.createElement("label");
    textValue.innerHTML = 'Position is null. ';
    
    // Post the UI into the Jupyter notebook cell

    el.appendChild(button);
    el.appendChild(textStatus);
    el.appendChild(textValue);
    
    // we include a return function to close the Phidget when the notebook is closed
    return closeUSB();
  }
    """
    _css = """
    .ph-button {color: white; 
        background-color:rgb(96, 107, 174); 
        border-radius: 8px; 
        font-size: 24px; 
        display: block;
        padding: 15px 32px;}
    .ph-button:hover { background-color:rgb(120, 128, 187); }  
    """
    value = traitlets.List(4*[0]).tag(sync=True) ## a list of 4 values, a quaternion

## Side note

At this point, you could use the widget with the one-line command 
> spatial_widget()

However, let's get creative and use Python code to read the device values and display on a ipywidget. 

In [3]:
## Create the spatial widget and link it to 4 sliders, representing 4 components of a quaternion.

phidget=spatial_widget()
gauge = [FloatProgress(min=-1,max=1,orientation='vertical',description = ['x','y','z','w'][i]) for i in range(4)]
def updateGauge(change):
    for i in range(4):
        gauge[i].value = change.new[i]

phidget.observe(updateGauge, names=["value"])

In [5]:
## Display the spatial widget and the four sliders. 

display(phidget,Box(gauge))

spatial_widget(value=[0, 0, 0, 0])

Box(children=(FloatProgress(value=0.0, description='x', max=1.0, min=-1.0, orientation='vertical'), FloatProgr…

In [20]:
## The sum of the squares of those 4 values should always add to one. Try running this cell when phidget is connected.
sum([phidget.value[i]**2 for i in range(4)])

0.9999999616693527

## Instructions

Use the mouse to click on the "Click to connect" button above. 

If the phidget does not connect right way, try clicking again. Also, verify that you have the phidget hub connected to your computer's USB port, and the phidget device is plugged into the hub. Once connected, you will see the device data values updates every second or so. 

When you are done, click on the button now labeled "Click to disconnect."

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)